In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import seaborn as sns
import imblearn 
import random

from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

In [2]:
df = pd.read_csv('sample-features-ip2.csv')

In [3]:
cols = ['IP Address', 'Is Attack IP', 'Login Successful',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'bot', 'desktop',
       'mobile', 'tablet']
df_features = df[cols]
df_features['Is Attack IP'] = df_features['Is Attack IP'].astype(int)
df_features = df_features.fillna(0)

/var/folders/dc/xfskm7hx35db80zpm6534xzr0000gn/T/ipykernel_1095/2790961587.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['Is Attack IP'] = df_features['Is Attack IP'].astype(int)


In [4]:
df_features[df_features.isna().any(axis=1)]

,IP Address,Is Attack IP,Login Successful,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,...,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,bot,desktop,mobile,tablet


In [5]:
df_features['Is Attack IP'].value_counts()

0    595766
1     17390
Name: Is Attack IP, dtype: int64

In [6]:
X = df_features.iloc[:, 2:].to_numpy()
y = df_features['Is Attack IP']

# from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
smote = imblearn.over_sampling.SMOTE(sampling_strategy=0.3)

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X, y)

print('Original dataset shape', y.value_counts())
print('Resample dataset shape', y_smote.value_counts())

Original dataset shape 0    595766
1     17390
Name: Is Attack IP, dtype: int64
Resample dataset shape 0    595766
1    178729
Name: Is Attack IP, dtype: int64


In [7]:
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegression

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = df_features.iloc[:, 2:]
y = df_features['Is Attack IP']

for fold, (tr_ind, val_ind) in enumerate(kfold.split(x_smote, y_smote)):
    X_train, X_val = x_smote[tr_ind], x_smote[val_ind]
    y_train, y_val = y_smote[tr_ind], y_smote[val_ind]
    cls = LogisticRegression()
    cls.fit(X_train, y_train)
    val_pred1 = cls.predict(X_val)
    print('Accuracy score -', fold+1, ': ', accuracy_score(y_val, val_pred1))


/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score - 1 :  0.7778036010561721


/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score - 2 :  0.7786945041607757


/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score - 3 :  0.7789914718623103


/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score - 4 :  0.7785718435884028
Accuracy score - 5 :  0.7786170343255928


/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
preds = cls.predict_proba(X)
preds = [1 if pred[1] > 0.30 else 0 for pred in preds]
confusion_matrix(y, preds)

/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[589605,   6161],
       [ 15660,   1730]])

In [12]:
X

,Login Successful,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,bot,desktop,mobile,tablet
0,0.670732,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,11.0,...,16.0,7.0,15.0,8.0,2.0,0.0,0.0,32.0,126.0,6.0
1,0.336957,0.0,1.0,14.0,6.0,3.0,2.0,4.0,1.0,4.0,...,7.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,91.0,0.0
2,0.818182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0
3,0.239005,24.0,65.0,94.0,129.0,157.0,165.0,148.0,149.0,176.0,...,67.0,105.0,33.0,46.0,29.0,20.0,1.0,3.0,2760.0,9.0
4,0.222785,3.0,3.0,10.0,8.0,9.0,11.0,23.0,24.0,12.0,...,17.0,27.0,10.0,15.0,21.0,9.0,0.0,2.0,392.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613151,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
613152,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
613153,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
613154,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [14]:
df_features

,IP Address,Is Attack IP,Login Successful,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,...,hour_19,hour_20,hour_21,hour_22,hour_23,bot,desktop,mobile,tablet,attack_probability
0,10.0.65.171,0,0.670732,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,7.0,15.0,8.0,2.0,0.0,0.0,32.0,126.0,6.0,0.829803
1,194.87.207.6,0,0.336957,0.0,1.0,14.0,6.0,3.0,2.0,4.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,91.0,0.0,0.778712
2,81.167.144.58,0,0.818182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0,0.833447
3,170.39.78.152,0,0.239005,24.0,65.0,94.0,129.0,157.0,165.0,148.0,...,105.0,33.0,46.0,29.0,20.0,1.0,3.0,2760.0,9.0,0.149450
4,10.0.0.47,1,0.222785,3.0,3.0,10.0,8.0,9.0,11.0,23.0,...,27.0,10.0,15.0,21.0,9.0,0.0,2.0,392.0,0.0,0.718625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613151,103.129.76.68,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.732822
613152,185.141.194.53,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.732822
613153,51.175.135.87,0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.851547
613154,192.30.85.191,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.730031


In [15]:
df_features['attack_probability'] = 0
df_features['attack_probability'] = cls.predict_proba(X)

/Users/nickbuchert/miniforge3/envs/en685648/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [20]:
df_features.to_csv('attack_preds.csv', index=False)

In [ ]:
df_features['total_logins'] = np.sum(df_features.iloc[:, 4:27], axis=1)

In [ ]:
df_features['total_logins'].clip(0,20).hist()